In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os, sys, glob
import gzip
import random
import tqdm
import json
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)

from IPython import display
import matplotlib as mpl
from matplotlib import pyplot as plt

%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
from google.colab import drive
drive.mount('/content/drive/') 


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [212]:
DATA_PATH = '/content/drive/MyDrive/' # '<insert-your-training-data-path-here>'

ROUND = 3 # This project will have 3 rounds of predictions: 1,2,3
STUDENT_ID = '28153'#'<insert-your-id-here>'
PROJECT_CODE = 'CS412cc29ccbb5807' #'<insert-your-code-here>' # Same code for the annotation eg. CS412xxxxx

In [87]:
#trainingTweetDf = pd.read_csv('{}training-tweet.csv'.format(DATA_PATH))
trainingTweetDf = pd.read_csv(DATA_PATH + 'training-tweet.csv', dtype={'tweet_id': str, 'isPolitical': str})
trainingTweetDf

,tweet_id,isPolitical
0,1597170281545551872,Yes
1,1431700027471192069,No
2,1566035577090281472,Yes
3,1591538690869940225,Yes
4,1583898169238167554,Yes
...,...,...
2995,1593539327623151619,Yes
2996,1393886554062524418,No
2997,1597925615092764672,Yes
2998,1585291418616176640,Yes


In [88]:
trainingTweetDf.isPolitical.value_counts()

Yes    2003
No      997
Name: isPolitical, dtype: int64

In [89]:
trainingUserDf = pd.read_csv('{}training-user.csv'.format(DATA_PATH))
#trainingUserDf = pd.read_csv('training-user.csv')
trainingUserDf

,screen_name,isBot
0,koftecancaddy,No
1,ahaber,No
2,selahat03949652,No
3,erdin06357062,No
4,bhct__necatii,No
...,...,...
2995,djblumenberg,No
2996,mel1sq,No
2997,eren_yz1,Yes
2998,ergnyildiz4,No


In [90]:
trainingUserDf.isBot.value_counts()

No     2424
Yes     576
Name: isBot, dtype: int64

In [91]:
# You can also expand training data by downloading your own labeled datasets following the link
# Download the documents under "Link to training data"

print('http://www.onurvarol.com/Annotation-CS412-202201/reports/report_{}.html'.format(PROJECT_CODE))

http://www.onurvarol.com/Annotation-CS412-202201/reports/report_CS412418a480522fd.html


In [123]:
#     FUNCTIONS TO EXTRACT DATA FROM TWEET_METADATA

list_of_entities = ['🇹🇷','recep', 'tayyip', 'erdoğan', 'ak', 'parti', 'i̇lçe', 'başkan', 'yardımcısı', 'emniyetgm', 'adil', '@kilicdarogluk' , '@meral_aksener'
                        'tanıtım', 'medya', 'başkanı', 'yönetim',  'tayyip', 'genel', 'haber',  'başkanlığı', 'müdürlüğü',   'merkez', 'teşkilat', 'i̇l',  'emekli', 'belediye',  'kanun', 'mağduru',   'üyesi',  'kadın', 'kolları',   'kurucu', 'parti̇','milli', 'anadolu', 'gençlik',  'chp', 'başkanı.', 'kurulu',  'sayılı', 'kemal',  'siyaset',  'vatan',  'partisi',  'milletvekili', 'atatürkçü',  'mustafa', 'atatürk', 'müslüman', 'allah',  'mhp', 'başkani',  'bakanlığı', 'üyesi̇',  'devlet', 'türküm', 'mutlu',  'ülkü', 'atama', 'kadin', 'yönetimi', 'milliyetçi', 'il', 'meclis',  'kollari', '#parmağımdelinmesin', '@anadoludiyabet',  'güncel', 'ekonomi', 'ilçe', 'siyasi', 'gündem',  'ocakları',  'akparti', 'belediyesi','zafer',
                        'Meral',  'Sayın',  'Türk', 'Sn.', 'CHP', 'Süleyman',  'Mansur',  'Cumhurbaşkanı', 'Erdoğan', 'milyar', '128', 'suriyeli', 'göçmen', 'suriye', 'afgan', 'paki', 'kaçak', 'sığınmacı', 'mülteci', 'milyon',  'Recep', 'Tayyip',  'iş', 'istiyoruz', 'Belediye', 'Ekrem', 'destek', 'Sağlık', 'Toplantısı',  'Bakanımız', '@is1234tan1',  '@BY', 'kavala', 'gezi',  'Cumhurbaşkanımız', 'hak', 'Kılıçdaroğlu,', 'bakanım', 'Allah', 'adam',  'Bakanı',  'Başkanımız',  'hizmet',  'İmamoğlu',  'Türkiye', 'mücadele', 'bekleyen', 'Kemal', 'Kılıçdaroğlu',  'yangın', 'Devlet', 'Sn', 'ABD', 'Ne', '@is1234tan1:', '@mehmedmus', '@tcmeb', 'yerine', 'TAYYİP',  'Cumhurbaşkanlığı', 'orman','AK', 'Milli',   'atama',  'AF',   '15', 'öğretmen',  'İl',  'ülke', 'Akşener', 'İBB', 'sağlık',  'sayın', 'kabul',  'Bahçeli',  'Açılış', 'yol',  'Yavaş', 'ekonomi', 'dolar', 'döviz', 'faiz', 'tüik', 'enflasyon', 'zam', 'yağ', 'un', 'çiftçi', 'mazot', 'motorin', 'benzin', 'FED', 'tcmb', 'banka', 'aşı', 'asker', 'eyt', 'EYT', 'mağdur', 'magdur', 'TBMM', 'dolar',
                        'atama', 'euro', 'melihgökçek', 'erkenseçim', 'bakan', 'bakanlık', 'barışatay', 'hdp', 'ekremimamoglu', 'yenisafak', 'umitozdag', 'dbdevletbahceli', 'rterdogan', 'suleymansoylu', 'arsivunutmaz', 'drfahrettinkoca', 'meral_aksener', 'kilicdarogluk', '06melihgokcek', 'murat_kurum', 'mevlutcavusoglu', 'akparti', 'medyaadami', 'bekirpakdemirli', 'is1234tan1', 'mansuryavas06', 'tc_icisleri', 'kasapoglu', 'azpresident', 'alpayozalan35', 'goksal_islam', '06afsar', 'erkankandemir', 'mhp_bilgi', 'tcbestepe', 'herkesicinchp', 'akaraismailoglu', 'iletisim', 'rtedijital', 'meraluz15384260', 'fahrettinaltun', 'hikmetyt1', 'tcbuyuksehir', 'varank', 'slmhktn', 'deryayanikashb', 'akkadinistanbul', 'ayyildirim1', 'ibbhabercomtr', 'alcayircengiz', 'akanomer71', 'mediamuhtari']

def check_if_retweet(tweet_metadata_line):
    is_retweet = 0
    retweeted_username = None

    try:
        tweet_metadata_line['retweeted_status']
        retweeted_username = tweet_metadata_line['retweeted_status']['user']['screen_name'].lower()
        is_retweet = 1

    except KeyError:
        pass

    return is_retweet, retweeted_username

def retweeted_from_politician(retweeted_username):
    RFP = 1 #False
    if retweeted_username in acc_names:
      RFP = 0
    return RFP

def get_tweet_text(tweet_metadata_line):
    text = tweet_metadata_line['text']
    
    return text , len(text)

def get_tweet_id(tweet_metadata_line):
    id_str = tweet_metadata_line['id_str']
    
    return id_str

def get_number_mentions_hashtags(tweet_metadata_line):
    num_mentions = len(tweet_metadata_line['entities']['user_mentions'])
    num_hashtags = len(tweet_metadata_line['entities']['hashtags'])

    return num_mentions, num_hashtags

def get_number_retweets_favorites(tweet_metadata_line):
    retweet_count = tweet_metadata_line['retweet_count']
    favorite_count = tweet_metadata_line['favorite_count']
    
    return retweet_count, favorite_count

def get_user_info(tweet_metadata_line):
    id = tweet_metadata_line['user']['id_str']
    screen_name = tweet_metadata_line['user']['screen_name'].lower()
    description = tweet_metadata_line['user']['description']

    return id, screen_name, description

def check_political_ent(text):
    for i in range(len(list_of_entities)):
      list_of_entities[i] = list_of_entities[i].lower()
      
    entities_in_text = [ent for ent in list_of_entities if ent in text.lower()]
    number_entities = len(entities_in_text)

    return number_entities

def total_interactions(retweet_count, favorite_count):
    total_num_interactions = retweet_count + favorite_count
    
    return total_num_interactions


def get_retweet_tweet_ratio(line):
    number_retweets = 0
    number_original_tweets = 0

    for tweet in line['tweets']:
        try:
            tweet['retweeted_status']
            number_retweets += 1
                
        except:
            number_original_tweets += 1
            
    total_tweets = number_retweets + number_original_tweets
    
    if total_tweets == 0:
        retweet_total_ratio = None
    else:
        retweet_total_ratio = number_retweets/(total_tweets)
    
    return retweet_total_ratio


def get_total_tweet_num(line):
    number_retweets = 0
    number_original_tweets = 0

    for tweet in line['tweets']:
        try:
            tweet['retweeted_status']
            number_retweets += 1
                
        except:
            number_original_tweets += 1

    total_tweets = number_retweets + number_original_tweets 

    return total_tweets

def get_replied_usernames(tweet_metadata_line):

  dictionary = {}
  for item in tweet_metadata_line["entities"]["user_mentions"]:
    if item["screen_name"] not in dictionary.keys:
      dictionary["screen_name"] = 1
    else:
      dictionary["screen_name"] += 1
  return dictionary


In [124]:
#      FEATURE EXTRACTION FROM TWEETS_METADATA

dfPolitical = {'tweet_id':[],
              'is_retweet':[],
              'retweeted_username':[],
              'text':[],
              'num_mentions':[],
              'num_hashtags':[],
              'num_retweets':[],
              'num_favorites':[],
              'user_id':[],
              'user_screen_name':[],
              'user_description':[],
              'num_political_entities':[],
              'total_interactions':[],
              'text_len':[],
              'longText':[],
              'description_political_num':[],
              'RFP':[]
              }

with gzip.open(f"{DATA_PATH}tweet_metadata.jsons.gz", "rb") as f:
    for line in f:
        line = json.loads(line)
        
        # raw data:
        id_str = get_tweet_id(line)
        is_retweet, retweeted_username = check_if_retweet(line)
        text, text_len = get_tweet_text(line)
        num_mentions, num_hashtags = get_number_mentions_hashtags(line)
        retweet_count, favorite_count = get_number_retweets_favorites(line)
        user_id_str, screen_name, user_description = get_user_info(line)

        # manually crafted data:
        num_political_entities = check_political_ent(text) 
        total_num_interactions = total_interactions(retweet_count, favorite_count)

        dfPolitical['tweet_id'].append(id_str)
        dfPolitical['is_retweet'].append(is_retweet)
        dfPolitical['retweeted_username'].append(retweeted_username)
        dfPolitical['text'].append(text)
        dfPolitical['num_mentions'].append(num_mentions)
        dfPolitical['num_hashtags'].append(num_hashtags)
        dfPolitical['num_retweets'].append(retweet_count)
        dfPolitical['num_favorites'].append(favorite_count)
        dfPolitical['user_id'].append(user_id_str)
        dfPolitical['user_screen_name'].append(screen_name)
        dfPolitical['user_description'].append(user_description)
        dfPolitical['num_political_entities'].append(num_political_entities)
        dfPolitical['total_interactions'].append(total_num_interactions)
        dfPolitical['text_len'].append(text_len)
        dfPolitical['description_political_num'].append(check_political_ent(user_description))
        dfPolitical['RFP'].append(retweeted_from_politician(retweeted_username))

        if text_len > 200:
          dfPolitical['longText'].append(1)
        
        else:
            dfPolitical['longText'].append(0)

In [125]:
#  TABLE OF TWEET FEATURES | POLITICAL / NOT 
dfPolitical = pd.DataFrame(dfPolitical)
dfPolitical

,tweet_id,is_retweet,retweeted_username,text,num_mentions,num_hashtags,num_retweets,num_favorites,user_id,user_screen_name,user_description,num_political_entities,total_interactions,text_len,longText,description_political_num,RFP
0,1588568792984346624,0,None,"Sosyal Hizmetin temelini çocuk oluşturur,çocuğ...",0,0,49,98,920963718103650304,maviruh_,shu/\nburaya afilli bir söz yazdığımı varsayın,3,147,140,0,4,1
1,1588452263047069697,0,None,"@mahirunal Gavur İzmir ya onlar, hani Cumhuriy...",1,0,0,0,595514060,mtfdan,,2,0,97,0,0,1
2,1569589330544398336,0,None,#ŞehitAdayıUzmÇvşaKadro\nSiz İstesenizde Istem...,0,1,0,0,1356375754561490947,ahsucilginuzman,Vatan Sevdalisi,2,0,140,0,1,1
3,1570428119609139201,0,None,@ajans_muhbir Siz kaypak olmayıp onay vermesey...,1,0,0,0,1478775431008595968,hamitelkelle,HighOne,3,0,140,0,1,1
4,1551163840368414722,0,None,Engelli öğretmenler olarak önümüzdeki engeller...,0,0,0,0,1511976696337113088,sed58417690,,3,0,140,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33528,1568595408233832448,0,None,Gerçek kimlik taşımayan hesaplara cevap vermem...,0,0,9,81,576247173,ardanzenturk,RT ONAYLADIĞIM ANLAMINA GELMEZ\nArtık fikirler...,3,90,140,0,2,1
33529,1584027427696959488,0,None,@umitozdag Neden Suriyelilerle ilgili bu kadar...,1,0,1,8,162308585,ozgul_61,Bridge design engineer Yaay hesabı : dilfiruz,9,9,140,0,2,1
33530,1585945783307730945,0,None,@celebimehmeta Niye Türkiye yüzyılıda.Türkiye ...,1,0,0,1,415025519,ladrekova,,3,1,76,0,0,1
33531,1569748909521801221,1,muazzezeralp,RT @muazzezeralp: @Doan58213655 @denizkonur @N...,7,1,6,0,1442125177727307781,yapikytgrivrlsn,,5,6,140,0,0,1


In [126]:
pol_names = {}
for i in dfPolitical['retweeted_username']:
  if i not in pol_names.keys():
    pol_names[i] = 1
  else:
    pol_names[i] += 1

print(pol_names)

{None: 28921, 'goktsibrahim': 1, 'mehmetsekmen': 2, 'serbestiyetweb': 1, 'bizbeklerizz': 2, 'ibbhabercomtr': 7, 'kilicdarogluk': 86, 'suleymansoylu': 100, 'umitozdag': 180, 'akparti': 94, 'varank': 32, 'zeynepimamoglu': 2, 'abdulkerimsevik': 1, 'ekrem_imamoglu': 39, 'ihsankoca44': 5, 'colcicegimm': 2, 'misvakcaps': 10, 'rterdogan': 467, 'camdakidero': 2, 'ozgesaimkaya': 1, 'gkkyhakan': 1, '06melihgokcek': 94, '1_omer__tanas__': 1, 'a82317335aslan': 3, 'yenisafak': 9, 'sonsuzask____': 3, 'rukycmn': 2, 'murat_kurum': 18, 'nur3485': 1, 'iletisim': 16, 'mhp_bilgi': 9, 'blablab00': 6, 'kasapoglu': 27, 'abdullahciftcib': 10, 'ozan_blk07': 5, 'whisperhaber': 2, '__huriye__59__': 1, 'atanama08092864': 3, 'mamili70406025': 1, 'kursadbilginci': 1, 'selin032623301': 1, 'baskansayan04': 1, 'sukokdr': 1, 'mesutayaz02': 1, 'furkanyildiz75': 1, 'akaraismailoglu': 43, 'kartalakparti': 1, 'eacarer': 2, 'ozturkazatt': 2, 'gozal_ensar': 1, 'kutlu4232': 1, 'ahmetasmink': 2, 'metinar69040971': 1, 'cetin_er

In [127]:
account_names = []
for i,j in pol_names.items():
  if j > 20:
    account_names.append(i)

#print(account_names)

acc_names = []
for a in account_names:
  if a is not None:
    acc_names.append(a)

print(acc_names)

['kilicdarogluk', 'suleymansoylu', 'umitozdag', 'akparti', 'varank', 'ekrem_imamoglu', 'rterdogan', '06melihgokcek', 'kasapoglu', 'akaraismailoglu', 'tcbestepe', 'meral_aksener', 'mansuryavas06']


In [128]:
tweet_word_dict = {}
for word in dfPolitical['text']:
  for w in word.split(' '):
    if w not in tweet_word_dict.keys():
      tweet_word_dict[w] = 1
    else:
      tweet_word_dict[w] += 1

most_used = []
for i,j in tweet_word_dict.items():
  if j > 100 and i in list_of_entities:
    most_used.append(i);

print(most_used)

['hak', '@kilicdarogluk', 'milyon', 'ne', 'istiyoruz', 'sayın', '15', 'hizmet', 'atama', 'öğretmen', 'kabul', 'adam', 'yerine', 'destek', 'parti', 'iş', '🇹🇷', 'adil', 'mağdur', '@tcmeb', 'bakanım', 'devlet', 'sağlık']


In [171]:
#       FUNCTIONS TO EXTRACT METADATA FROM USER_PROFILES 

def get_user_info_metadata(user_metadata_line):
    
    user_id = user_metadata_line['id_str']
    user_name = user_metadata_line['name']
    user_screen_name = user_metadata_line['screen_name'].lower()
    user_location = user_metadata_line['location']
    user_description = user_metadata_line['description']
    user_followers_count = user_metadata_line['followers_count']
    user_friends_count = user_metadata_line['friends_count']
    user_verified_info = user_metadata_line['verified']
    user_statuses_count = user_metadata_line['statuses_count']
    user_is_protected = user_metadata_line['protected']
    user_listed_count = user_metadata_line['listed_count']
    
    
    dictionary = {'user_id':user_id, 'user_name': user_name, 'user_screen_name':user_screen_name, 'user_location':user_location,
     'user_description':user_description, 'user_followers_count':user_followers_count, 'user_friends_count':user_friends_count, 'user_verified_info': user_verified_info, 'user_statuses_count': user_statuses_count, 'user_is_protected': user_is_protected, 'user_listed_count': user_listed_count}

    return dictionary
    
def get_followers_all_ratio(user_followers_count, user_friends_count):
    
    if user_friends_count + user_followers_count == 0:
        followers_all_ratio = 0

    else:
        followers_all_ratio =  user_followers_count / (user_friends_count + user_followers_count)

    return followers_all_ratio

def get_desc_len(user_description):
    
    description_len = len(user_description)

    return description_len
    
def get_verified_status(verified):
    if verified == False:
      verified_status = 0
    else:
      verified_status = 1
    return verified_status

def check_is_protected(protected):
    if protected == False:
      protected_status = '0'
    else:
      protected_status = '1'
    return protected_status


In [172]:
dfBot = {'user_id':[],
         'user_name':[],
         'user_screen_name':[],
         'user_location':[],
         'user_description':[],
         'user_followers_count':[],
         'user_friends_count':[],
         'description_len':[],
         'followers_to_all_ratio':[],
         'user_verified_info':[],
         'user_verified_status':[],
         'user_statuses_count':[],
         'user_is_protected': [],
         'check_is_protected':[],
         'user_listed_count':[]}

with gzip.open(f"{DATA_PATH}user_profiles.jsons.gz", "rb") as f:
    for line in f:
        line = json.loads(line)

        dictionary = get_user_info_metadata(line)
        for k,v in dictionary.items():
            dfBot[k].append(v)

        
        # manually crafted data:
        description_len = get_desc_len(dictionary['user_description'])
        dfBot['description_len'].append(description_len)
        
        followers_all_ratio = get_followers_all_ratio(dictionary['user_followers_count'], 
                                                      dictionary['user_friends_count'])

        dfBot['followers_to_all_ratio'].append(followers_all_ratio)

        verified_status = get_verified_status(dictionary['user_verified_info'])
        dfBot['user_verified_status'].append(verified_status)

        user_is_protected_status = check_is_protected(dictionary['user_is_protected'])
        dfBot['check_is_protected'].append(user_is_protected_status)

In [173]:
# TABLE OF USER FEATURES | BOT / NOT
dfBot = pd.DataFrame(dfBot)
dfBot

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,description_len,followers_to_all_ratio,user_verified_info,user_verified_status,user_statuses_count,user_is_protected,check_is_protected,user_listed_count
0,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,7,0.260000,False,0,2551,False,0,0
1,1304340303080386560,fania :((((,scorpiehoez,bogor,have a holly jolly🎄,8235,3011,19,0.732260,False,0,42771,False,0,11
2,1116042038577958914,Yusuf Aksoy,yusufak63712920,,"Bir şeyden pişmanlık duymak istemiyorsan,her ş...",95,399,64,0.192308,False,0,14300,False,0,0
3,4859899931,Be (VIXX6) ama oppalarının düğününe gidemiyor,nedenburdaysam,Hufflepuff ortak salon,"SMStan\n/St☆rlight ///come on girls,this is ou...",40,83,65,0.325203,False,0,21303,False,0,5
4,2225373636,SLMDMR,biologselim,,BİYOLOG🔬🦠\nNanoteknoloji,100,98,23,0.505051,False,0,1629,False,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29665,1320834618220781569,islammm,islam_mrsj,,,229,217,0,0.513453,False,0,2396,False,0,0
29666,111074128,Melda Onur,meldaonur,,"Şekersiz çay, etsiz sofra, SAVAŞSIZ dünya... 🐌...",212457,5428,100,0.975088,False,0,75178,False,0,279
29667,36946875,ali ydm,ali_ydm,"İstanbul, Türkiye",hayat oyunsa bende jeton çok,116,141,28,0.451362,False,0,6482,False,0,0
29668,2389587396,Türkan Usta,turkanusta,"Ankara, Türkiye",Ustaya sormuşlar; hayatta yaptığın en büyük is...,1669,3639,116,0.314431,False,0,121113,False,0,1


In [174]:
# FEATURE EXTRACTION FUNCTIONS ABOUT TWEET INFO OF USERS:

def get_median_number_favorites(line):
    num_median_favorites = np.median([tweet['favorite_count'] for tweet in line['tweets']])

    return num_median_favorites

from datetime import datetime
import time 

def time_since_creation(line):
  creation_date = line['created_at']
  pos = creation_date.find("+")
  new_date = creation_date.substr(0,pos) + creation_date.substr(pos+5)

  new_date = datetime.strptime(new_date,"%a %b %d %H:%M:%S %Y")
  today = datetime.today()


  return (today-new_date).days()

def daily_tweet_ratio(line):
  tooMuchTweet = 1 #false
  tweet_per_day = get_total_tweet_num(line) / time_since_creation(line)
  if tweet_per_day > 25:
    tooMuchTweet = 0 #true
  return tooMuchTweet

def get_median_number_hashtags(line):
    num_median_hashtags = np.median([len(tweet['entities']['hashtags']) for tweet in line['tweets']])

    return num_median_hashtags

In [177]:
#       FEATURE EXTRACTION FROM USER_TWEETS 

dfBotTweets = {'user_id':[],
               'retweet_total_ratio':[],
               'num_median_favorites':[],
               'num_of_tweets':[],
               'num_median_hashtags':[],
              }

with gzip.open(f"{DATA_PATH}user_tweets.jsons.gz", "rb") as f:
    counter = 0
    for line in f:
        
        line = json.loads(line)

        user_id = line['user_id']
        dfBotTweets['user_id'].append(user_id)
        
        retweet_total_ratio = get_retweet_tweet_ratio(line)
        dfBotTweets['retweet_total_ratio'].append(retweet_total_ratio)
        
        total_tweet= get_total_tweet_num(line)
        dfBotTweets['num_of_tweets'].append(total_tweet)

        num_median_favorites = get_median_number_favorites(line)
        dfBotTweets['num_median_favorites'].append(num_median_favorites)
        
        num_median_hashtags = get_median_number_hashtags(line)
        dfBotTweets['num_median_hashtags'].append(num_median_hashtags)


        #too_much_Tweets = daily_tweet_ratio(line)
        #dfBotTweets['tooMuchTweet'].append(too_much_Tweets)

      
    print("All lines are scanned.")

/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


All lines are scanned.


In [179]:
dfBotTweets = pd.DataFrame(dfBotTweets)
dfBotTweets.sort_values("user_id")

,user_id,retweet_total_ratio,num_median_favorites,num_of_tweets,num_median_hashtags
1877,1000019437,0.186869,1.0,198,0.0
26488,1000036861832761345,0.016575,1.0,181,0.0
6428,1000054552652058624,0.655000,0.0,200,0.0
3750,1000061352726224896,0.775000,0.0,200,0.0
13347,1000095743103897600,0.160000,38.0,200,0.0
...,...,...,...,...,...
23254,999749026663804930,0.335000,1.0,200,0.0
16749,999759826518528000,0.435000,8.0,200,0.0
21630,999884985099210752,0.205000,0.0,200,0.0
9258,999936891217698816,0.020000,4.0,200,0.0


In [180]:
# MERGING ALL DATA WE HAVE SO FAR | USER STATISTICS
dfBotAll = dfBot.merge(dfBotTweets,
                       how='left')

dfBotAll[['retweet_total_ratio', 'num_median_favorites','num_median_hashtags', 'num_of_tweets']] = dfBotAll[['retweet_total_ratio', 'num_median_favorites','num_median_hashtags','num_of_tweets']].fillna(0)

dfBotAll

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,description_len,followers_to_all_ratio,user_verified_info,user_verified_status,user_statuses_count,user_is_protected,check_is_protected,user_listed_count,retweet_total_ratio,num_median_favorites,num_of_tweets,num_median_hashtags
0,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,7,0.260000,False,0,2551,False,0,0,0.395939,0.0,197.0,0.0
1,1304340303080386560,fania :((((,scorpiehoez,bogor,have a holly jolly🎄,8235,3011,19,0.732260,False,0,42771,False,0,11,0.125000,0.0,200.0,0.0
2,1116042038577958914,Yusuf Aksoy,yusufak63712920,,"Bir şeyden pişmanlık duymak istemiyorsan,her ş...",95,399,64,0.192308,False,0,14300,False,0,0,0.910000,0.0,200.0,1.0
3,4859899931,Be (VIXX6) ama oppalarının düğününe gidemiyor,nedenburdaysam,Hufflepuff ortak salon,"SMStan\n/St☆rlight ///come on girls,this is ou...",40,83,65,0.325203,False,0,21303,False,0,5,0.015306,1.0,196.0,0.0
4,2225373636,SLMDMR,biologselim,,BİYOLOG🔬🦠\nNanoteknoloji,100,98,23,0.505051,False,0,1629,False,0,0,0.659898,0.0,197.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29665,1320834618220781569,islammm,islam_mrsj,,,229,217,0,0.513453,False,0,2396,False,0,0,0.015000,1.0,200.0,0.0
29666,111074128,Melda Onur,meldaonur,,"Şekersiz çay, etsiz sofra, SAVAŞSIZ dünya... 🐌...",212457,5428,100,0.975088,False,0,75178,False,0,279,0.291457,2.0,199.0,0.0
29667,36946875,ali ydm,ali_ydm,"İstanbul, Türkiye",hayat oyunsa bende jeton çok,116,141,28,0.451362,False,0,6482,False,0,0,0.061538,0.0,195.0,0.0
29668,2389587396,Türkan Usta,turkanusta,"Ankara, Türkiye",Ustaya sormuşlar; hayatta yaptığın en büyük is...,1669,3639,116,0.314431,False,0,121113,False,0,1,0.995000,0.0,200.0,0.0


DONE WITH THE FEATURE EXTRACTION PART!


In [181]:
# TRAINING THE MODEL

# 1. POLITICAL TWEET PREDICTION:

dfPoliticalAll_train = dfPolitical.merge(trainingTweetDf,
                                         on='tweet_id')

dfPoliticalAll_train.head(10)

,tweet_id,is_retweet,retweeted_username,text,num_mentions,num_hashtags,num_retweets,num_favorites,user_id,user_screen_name,user_description,num_political_entities,total_interactions,text_len,longText,description_political_num,RFP,isPolitical
0,1585955683513798656,0,None,@AvOzlemZengin YüzüncüYıla YakışanGenelAf adli...,1,0,3,2,1564992353168941058,zehra78231638,,6,5,126,0,0,1,Yes
1,1597631718479261696,0,None,#TCYüzyılıÜcretliÖgrtKadro\n#TCYüzyılıÜcretliÖ...,0,2,30,28,1324630334416297985,nurozguler,,3,58,140,0,0,1,Yes
2,1572522789948751874,0,None,Ekrem İmamoğlu davayı değerlendirdi. 'Boş işle...,0,0,5,66,407597071,onediocom,Türkiye'nin ilk ve tek sosyal içerik sitesi ht...,4,71,131,0,8,1,Yes
3,1591412481561624577,0,None,Sayın Bakanım @suleymansoylu POMEM önlisans er...,1,0,0,0,1394789887073738753,buckybarnestr,...,8,0,140,0,0,1,Yes
4,1596914274907348992,0,None,"@varank Sayın bakanım, Bodrumdaki bu araziyi ...",1,0,0,0,1586083256088371201,sayariahmet,,9,0,87,0,0,1,Yes
5,1597147910184968192,0,None,"@drfahrettinkoca Üniversite sınavında, KPSS de...",1,0,1,3,1595686009416605697,dytmyt2,Diyetisyen Etkinlik Hesabı,6,4,140,0,0,1,Yes
6,1596356027129401344,0,None,@HaberturkTV @filaligs1905 @ProfDrErsanSen Eğe...,3,0,0,2,3605839396,2121elyas,Allah bizi hak yolundan ayırmasın. Batıldan uz...,7,2,140,0,7,1,Yes
7,1407916468230967299,0,None,@bIuemarinewes 40k yu sini https://t.co/YRvYTh...,0,0,0,0,1398300266185695232,ad3www,akun ba berubah jadi akun pejuang GA,1,0,50,0,3,1,No
8,1592261577381756935,0,None,@dfikrisaglar Bıkmadın değilmi yalanlara,1,0,0,1,1558508220062306304,tulga343434,,1,1,40,0,0,1,Yes
9,1597125142119059456,0,None,"@alikenanoglu ... devri daim olsun, yıldızlar ...",1,0,0,0,141573197,camurdan_,"#sociology, #hamdolsunülkesi nde sıradan yaşam...",3,0,60,0,3,1,No


In [182]:
X = dfPoliticalAll_train[['num_political_entities','total_interactions','num_hashtags','num_mentions','text_len','description_political_num','num_retweets', 'RFP']]
X.rename(columns = {'num_political_entities':'f0', 'total_interactions':'f1','num_hashtags':'f2', 'num_mentions':'f3','text_len':'f4', 'description_political_num':'f5', 'num_retweets':'f6', 'RFP':'f7'}, inplace = True)

y = dfPoliticalAll_train['isPolitical'].apply(lambda x: 1 if x=='Yes' else 0)

# define X as we did above in section (2.x.2. Separate X and y values)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [183]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42)

In [184]:
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error


# create an instance
dtc_political = RandomForestClassifier(bootstrap='True', max_depth=20, max_features=2, min_samples_leaf=10, min_samples_split=10, n_estimators=1150, random_state=42)

# fit your model
dtc_political.fit(X_train, y_train)

# make predictions
preds = dtc_political.predict(X_valid)

# evaluate on validation set
acc_score = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)
mse = mean_squared_error(y_valid, preds)

print("MSE:", mse, "\n",
      "Accuracy Score:", acc_score, "\n",
      "Confusion Matrix:", "\n", confusion)


MSE: 0.245 
 Accuracy Score: 0.755 
 Confusion Matrix: 
 [[ 89 120]
 [ 27 364]]


In [185]:
dfBotAll = dfBotAll.dropna()
trainingUserDf = trainingUserDf.dropna()

In [186]:
# 2.BOT DETECTION:

dfBotAll.user_screen_name = dfBotAll.user_screen_name.str.lower()
dfBotAll_train = dfBotAll.merge(trainingUserDf,
                               left_on='user_screen_name',
                               right_on='screen_name')
dfBotAll_train

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,description_len,followers_to_all_ratio,user_verified_info,user_verified_status,user_statuses_count,user_is_protected,check_is_protected,user_listed_count,retweet_total_ratio,num_median_favorites,num_of_tweets,num_median_hashtags,screen_name,isBot
0,1512081815292432394,sezgin,sezgin953116371,,,46,430,0,0.096639,False,0,1535,False,0,0,0.050251,0.0,199.0,0.0,sezgin953116371,No
1,1425452291428077571,Adem Koç,gogoadem61,,,14,171,0,0.075676,False,0,113,False,0,0,0.761062,0.0,113.0,1.0,gogoadem61,No
2,328164303,Necmettin Balıkçı,dewil511,,,21,49,0,0.300000,False,0,219,False,0,1,0.010101,0.0,198.0,1.0,dewil511,Yes
3,1343666971368431622,Night Bird⁷🦉,midnight__bird,,"La vie est un sommeil, l’amour en est le rêve...",422,260,48,0.618768,False,0,15191,False,0,9,0.085000,1.0,200.0,0.0,midnight__bird,No
4,1240932880488038400,Samed Pınarcı,samedpinarci,,Orman Mühendisi - Orman İşletme Şefi - Orman G...,133,202,60,0.397015,False,0,3734,False,0,0,0.780000,0.0,200.0,0.0,samedpinarci,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,1286770207134973954,Hamide Arabacı,anka6054,,,151,61,0,0.712264,False,0,5288,False,0,0,0.000000,1.0,200.0,0.0,anka6054,No
2996,1598032338323214338,atamabekleyenbahceci,atamabekleyenzz,,,173,367,0,0.320370,False,0,311,False,0,0,0.580000,0.0,200.0,0.0,atamabekleyenzz,No
2997,760235343966863360,Emrah İNCİ,memrahinci,Istanbul - Bayburt,Researcher | Middle East | Political Science |...,5863,5905,71,0.498215,False,0,1029,False,0,0,0.040000,36.0,200.0,0.0,memrahinci,No
2998,1553973684100124672,Murat Kkk,muratkkk18,,Normal sıradan bir insanım,1,10,26,0.090909,False,0,18,False,0,0,0.769231,0.0,13.0,0.0,muratkkk18,No


In [187]:
trainingUserDf.isBot.value_counts()

No     2424
Yes     576
Name: isBot, dtype: int64

In [188]:
X = dfBotAll_train[['description_len', 'followers_to_all_ratio', 'retweet_total_ratio', 'num_median_favorites', 'check_is_protected', 'num_of_tweets', 'num_median_hashtags', 'user_friends_count', 'user_followers_count']]
y = dfBotAll_train.isBot.apply(lambda x: 1 if x=='Yes' else 0)

In [189]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42)

In [190]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error

# create an instance
dtc_bot = RandomForestClassifier(max_depth=10, max_features=2, min_samples_leaf=6, min_samples_split=5, n_estimators=1150, random_state=42)

# fit your model
dtc_bot.fit(X_train, y_train)

# make predictions
preds = dtc_bot.predict(X_valid)

# evaluate on validation set
acc_score = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)

print("MSE:", mse, "\n",
      "Accuracy Score:", acc_score, "\n",
      "Confusion Matrix:", "\n", confusion)

MSE: 0.245 
 Accuracy Score: 0.8333333333333334 
 Confusion Matrix: 
 [[479  14]
 [ 86  21]]


TRAINING PART IS DONE!

In [202]:
# MAKING PREDICTIONS WITH OUR MODEL:

# read the evaluation file as follows
evaluationTweetDf = pd.read_csv('{}evaluation-round3-tweet.csv'.format(DATA_PATH), dtype={0: str}, header=None, names=['tweet_id'])
evaluationTweetDf = evaluationTweetDf.dropna()
evaluationTweetDf

# merge it with the political dataframe so that you can use the make predictions based on the variables
dfPolitical_test = dfPolitical.merge(evaluationTweetDf)

# merge it with the political dataframe so that you can use the make predictions based on the variables
dfPolitical_test = dfPolitical.merge(evaluationTweetDf)

# define X as we did above in section (2.x.2. Separate X and y values)
X = dfPolitical_test[['num_political_entities','total_interactions','num_hashtags','num_mentions','text_len','description_political_num','num_retweets', 'RFP']]

# make predictions based on these variables
predictions_ispolitical = dtc_political.predict(X)

modelPredTweet = dict([(x,float(y)) for x,y in zip([*dfPolitical_test.tweet_id], predictions_ispolitical)])
modelPredTweet

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- RFP
- description_political_num
- num_hashtags
- num_mentions
- num_political_entities
- ...
Feature names seen at fit time, yet now missing:
- f0
- f1
- f2
- f3
- f4
- ...

  warnings.warn(message, FutureWarning)


{'1434787703783051264': 1.0,
 '1367571642604544000': 1.0,
 '1589993032975544320': 1.0,
 '1565312596135354373': 1.0,
 '1579558096833511424': 1.0,
 '1439547067337256967': 0.0,
 '1559963768372740098': 1.0,
 '1562853131251118081': 1.0,
 '1586021183958704128': 1.0,
 '1585766233491886081': 1.0,
 '1427746815420604417': 0.0,
 '1352635736537882629': 1.0,
 '1415032260571680768': 1.0,
 '1548636597628899328': 1.0,
 '1564926450096013313': 1.0,
 '1585634359612420101': 1.0,
 '1597138789108895744': 1.0,
 '1391681495622995971': 0.0,
 '1389951943343316995': 1.0,
 '1452348722810138646': 1.0,
 '1595829502021623812': 1.0,
 '1413108476348354562': 1.0,
 '1579408398894137344': 0.0,
 '1570758749606019073': 1.0,
 '1366091745772077058': 0.0,
 '1588936302086737925': 1.0,
 '1584922292127256577': 1.0,
 '1586279180983042050': 1.0,
 '1434963357023473671': 0.0,
 '1399687111234756612': 0.0,
 '1363226564683522051': 0.0,
 '1596978345560150016': 1.0,
 '1593686506622914560': 1.0,
 '1350478535954198530': 1.0,
 '157227522855

In [203]:
evaluationUserDf = pd.read_csv('{}evaluation-round3-user.csv'.format(DATA_PATH), dtype={0: str}, header=None, names=['user_screen_name'])
evaluationUserDf = evaluationUserDf.dropna()

# merge it with the political dataframe so that you can use the make predictions based on the variables
dfBot_test = dfBotAll.merge(evaluationUserDf)

# define X as we did above in section (2.x.2. Separate X and y values)
X = dfBot_test[['description_len', 'followers_to_all_ratio', 'retweet_total_ratio', 'num_median_favorites', 'check_is_protected', 'num_of_tweets', 'num_median_hashtags', 'user_friends_count', 'user_followers_count']]

# make predictions based on these variables
predictions_bot = dtc_bot.predict(X)

modelPredUser = dict([(x,float(y)) for x,y in zip([*dfBot_test.user_screen_name], predictions_bot)])
modelPredUser

{'biologselim': 0.0,
 'omerakdag34': 0.0,
 'bilgin21604923': 0.0,
 '_sydneycarton_': 0.0,
 'denizlihabercom': 0.0,
 'burakerbaychp': 0.0,
 'mvnez': 0.0,
 'qara118': 0.0,
 'nabiyonyevrum': 0.0,
 'farukhalit2': 0.0,
 'harlunoshi': 0.0,
 'heritagepaix': 0.0,
 'nuranwolf': 0.0,
 'politikgundem': 0.0,
 'isakethudax': 0.0,
 'enveraysevera': 0.0,
 'ilaydejaneiro': 0.0,
 '1905anason': 0.0,
 'eraydurgut03': 0.0,
 'dasiskein': 0.0,
 'ercan_bas29': 0.0,
 'mett_1907': 0.0,
 'ondemir066': 0.0,
 'semihyeteer': 0.0,
 'haberinyokcokk': 0.0,
 'meleky_ozaydin': 0.0,
 'mehmetaltay64': 0.0,
 'nurtencam2': 0.0,
 'bilobi4': 0.0,
 'yorumsuzadam87': 0.0,
 'twitsildiren': 0.0,
 '21gramlife1': 0.0,
 'cakan0_': 0.0,
 'oguzksalici': 0.0,
 'canpixlr': 0.0,
 'ozlemesor': 0.0,
 'emre_caliskann': 0.0,
 'oguzhanonel': 0.0,
 'mintchocolatemx': 0.0,
 'mehmet07454846': 0.0,
 'begumkarabeyx': 0.0,
 'imamgibiimam': 0.0,
 'durdane52': 0.0,
 'rk_ozanyali': 0.0,
 'suaterce': 0.0,
 'ibrahimcalsskan': 0.0,
 'goksubayy': 0.0,
 '

PREDICTION PART IS DONE!

In [206]:
# Explain your approach

data_explanations = '''
We used all of the datasets that is provided to us: 'annotation_users', 'annotation_tweets', 'tweet-data', 'evoluations3', 'training data'.
'''

feature_explanations = '''
Politics:
We created verified status and protected account features and checked these features:
['num_political_entities','total_interactions','num_hashtags','num_mentions','text_len','description_political_num','num_retweets', 'RFP'] 


BOT:
We created a daily tweet ratio feature and checked these features: 
['description_len', 'followers_to_all_ratio', 'retweet_total_ratio', 'num_median_favorites', 'check_is_protected', 'num_of_tweets', 'num_median_hashtags', 'user_friends_count', 'user_followers_count']]
'''

model_explanations = '''
We used Random Forest for increasing the accuracy. At each round, we will try to use different models such as K-Folds.  
We tried LogisticRegression model too, but it gave us a worse score of Accuracy, therefore we did not use it.
'''

additional_explanations = '''
We will try to consider words, dates, number of retweets in other models too. So we are planning to increase the total accuracy. 
'''

In [213]:
predictions = {
    'round': ROUND,
    'student_id': STUDENT_ID,
    'user_predictions': modelPredUser,
    'tweet_predictions': modelPredTweet,
    'explanations': {
        'data': data_explanations,
        'feature': feature_explanations,
        'model': model_explanations,
        'other': additional_explanations,
    }
}


with open('predictions-{}_round{}.json'.format(STUDENT_ID, ROUND), 'w') as fl:
    fl.write(json.dumps(predictions, indent=4))


In [214]:
# Test your submission file

submission = json.load(open('predictions-{}_round{}.json'.format(STUDENT_ID, ROUND), 'r'))
submission

{'round': 3,
 'student_id': '28153',
 'user_predictions': {'biologselim': 0.0,
  'omerakdag34': 0.0,
  'bilgin21604923': 0.0,
  '_sydneycarton_': 0.0,
  'denizlihabercom': 0.0,
  'burakerbaychp': 0.0,
  'mvnez': 0.0,
  'qara118': 0.0,
  'nabiyonyevrum': 0.0,
  'farukhalit2': 0.0,
  'harlunoshi': 0.0,
  'heritagepaix': 0.0,
  'nuranwolf': 0.0,
  'politikgundem': 0.0,
  'isakethudax': 0.0,
  'enveraysevera': 0.0,
  'ilaydejaneiro': 0.0,
  '1905anason': 0.0,
  'eraydurgut03': 0.0,
  'dasiskein': 0.0,
  'ercan_bas29': 0.0,
  'mett_1907': 0.0,
  'ondemir066': 0.0,
  'semihyeteer': 0.0,
  'haberinyokcokk': 0.0,
  'meleky_ozaydin': 0.0,
  'mehmetaltay64': 0.0,
  'nurtencam2': 0.0,
  'bilobi4': 0.0,
  'yorumsuzadam87': 0.0,
  'twitsildiren': 0.0,
  '21gramlife1': 0.0,
  'cakan0_': 0.0,
  'oguzksalici': 0.0,
  'canpixlr': 0.0,
  'ozlemesor': 0.0,
  'emre_caliskann': 0.0,
  'oguzhanonel': 0.0,
  'mintchocolatemx': 0.0,
  'mehmet07454846': 0.0,
  'begumkarabeyx': 0.0,
  'imamgibiimam': 0.0,
  'du